This demonstration has two parts:

1)  Climatology.make_climatology():
    This demonstration uses the COAsT package to calculate a climatological mean of an
    input dataset at a desired output frequency. Output can be written straight
    to file.

2) Climatology.make_multiyear_climatology():
    This demonstrations uses the COAsT package to calculate a climatological mean of an
    input dataset at a desired output frequency, over multiple years, but will work with single year datasets too.

COAsT and xarray should preserve any lazy loading and chunking. If defined
properly in the read function, memory issues can be avoided and parallel
processes will automatically be used.

In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [47]:
import coast
import glob
import xarray as xr
import cftime
import pandas as pd

### Usage of coast.Climatology.make_climatology().

Calculates mean over a given period of time. This doesn't take different years into account, unless using the
'years' frequency.

In [3]:
# root = "../../"
# # Paths to a single or multiple data files.
# dn_files = root + "./example_files/"
# fn_nemo_dat = dn_files + "coast_example_nemo_data.nc"
# fn_nemo_config = root + "./config/example_nemo_grid_t.json"
# # Set path for domain file if required.
# fn_nemo_dom = dn_files + "coast_example_nemo_domain.nc"
# # Define output filepath (optional: None or str)
# fn_out = None


In [4]:
data_path = "/gws/nopw/j04/canari/shared/large-ensemble/priority/HIST2/1"
fn_nemo_dom = "/gws/nopw/j04/canari/users/dlrhodso/mesh_mask.nc"
config_t = "../../config/example_nemo_grid_t.json"
infiles = glob.glob((f"{data_path}/OCN/yearly/*/*_votemper.nc"))

In [5]:
fn_nemo_dat = xr.open_mfdataset(infiles)

In [6]:
fn_nemo_dat

<xarray.Dataset> Size: 451GB
Dimensions:              (time_counter: 780, axis_nbounds: 2, deptht: 75,
                          y: 1207, x: 1442, nvertex: 4)
Coordinates:
  * time_counter         (time_counter) object 6kB 1950-01-16 00:00:00 ... 20...
  * deptht               (deptht) float32 300B 0.5058 1.556 ... 5.902e+03
  * y                    (y) float64 10kB 0.0 1.0 2.0 ... 1.205e+03 1.206e+03
  * x                    (x) float64 12kB 0.0 1.0 2.0 ... 1.44e+03 1.441e+03
    nav_lat              (y, x) float32 7MB dask.array<chunksize=(1207, 1442), meta=np.ndarray>
    nav_lon              (y, x) float32 7MB dask.array<chunksize=(1207, 1442), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds, nvertex
Data variables:
    time_counter_bounds  (time_counter, axis_nbounds) object 12kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    deptht_bounds        (time_counter, deptht, axis_nbounds) float32 468kB dask.array<chunksize=(12, 75, 2), meta=np.ndarray>
    bounds_nav_lat       (time_counter, y, x, nvertex) float32 22GB dask.array<chunksize=(12, 604, 721, 2), meta=np.ndarray>
    bounds_nav_lon       (time_counter, y, x, nvertex) float32 22GB dask.array<chunksize=(12, 604, 721, 2), meta=np.ndarray>
    votemper             (time_counter, deptht, y, x) float32 407GB dask.array<chunksize=(1, 19, 302, 361), meta=np.ndarray>
Attributes: (12/35)
    Conventions:            CF-1.10
    grid_label:             gn
    nominal_resolution:     25km
    realm:                  ocean
    contact:                
    parent_source_id:       CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.historical.r1i1p1f3
    ...                     ...
    title:                  [CANARI] GC3.1 N216 ORCA025 UM11.6
    license:                
    parent_experiment_id:   historical
    further_info_url:       https://github.com/NCAS-CMS/canari-data
    parent_activity_id:     
    external_variables:     areacello

In [8]:
nemo_data = coast.Gridded(fn_data=fn_nemo_dat,
                          fn_domain=fn_nemo_dom,
                          config=config_t,
                          ).dataset


In [9]:
nemo_data

<xarray.Dataset> Size: 453GB
Dimensions:              (t_dim: 780, axis_nbounds: 2, z_dim: 75, y_dim: 1207,
                          x_dim: 1442, nvertex: 4)
Coordinates:
  * time                 (t_dim) object 6kB 1950-01-16 00:00:00 ... 2014-12-1...
  * y                    (y_dim) float64 10kB 0.0 1.0 ... 1.205e+03 1.206e+03
  * x                    (x_dim) float64 12kB 0.0 1.0 2.0 ... 1.44e+03 1.441e+03
    longitude            (y_dim, x_dim) float32 7MB ...
    latitude             (y_dim, x_dim) float32 7MB ...
    depth_0              (z_dim, y_dim, x_dim) float64 1GB 0.5 0.5 ... 5.902e+03
Dimensions without coordinates: t_dim, axis_nbounds, z_dim, y_dim, x_dim,
                                nvertex
Data variables:
    time_counter_bounds  (t_dim, axis_nbounds) object 12kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    deptht_bounds        (t_dim, z_dim, axis_nbounds) float32 468kB dask.array<chunksize=(12, 75, 2), meta=np.ndarray>
    bounds_nav_lat       (t_dim, y_dim, x_dim, nvertex) float32 22GB dask.array<chunksize=(12, 604, 721, 2), meta=np.ndarray>
    bounds_nav_lon       (t_dim, y_dim, x_dim, nvertex) float32 22GB dask.array<chunksize=(12, 604, 721, 2), meta=np.ndarray>
    temperature          (t_dim, z_dim, y_dim, x_dim) float32 407GB dask.array<chunksize=(1, 19, 302, 361), meta=np.ndarray>
    bathymetry           (y_dim, x_dim) float64 14MB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    e1                   (y_dim, x_dim) float64 14MB ...
    e2                   (y_dim, x_dim) float64 14MB ...
    e3_0                 (z_dim, y_dim, x_dim) float64 1GB ...
    mask                 (z_dim, y_dim, x_dim) int8 131MB ...
    bottom_level         (y_dim, x_dim) int16 3MB ...
Attributes: (12/35)
    Conventions:            CF-1.10
    grid_label:             gn
    nominal_resolution:     25km
    realm:                  ocean
    contact:                
    parent_source_id:       CMIP6.CMIP.MOHC.HadGEM3-GC31-MM.historical.r1i1p1f3
    ...                     ...
    title:                  [CANARI] GC3.1 N216 ORCA025 UM11.6
    license:                
    parent_experiment_id:   historical
    further_info_url:       https://github.com/NCAS-CMS/canari-data
    parent_activity_id:     
    external_variables:     areacello

Calculate the climatology for temperature and sea surface height (ssh) as an example:

In [10]:
# Select specific data variables.
data = nemo_data[["temperature"]]

# Define frequency -- Any xarray time string: season, month, etc
climatology_frequency = "month"

In [11]:
# Calculate the climatology and write to file.
clim = coast.Climatology()
clim_mean = clim.make_climatology(data, climatology_frequency, fn_out=None)

Below shows the structure of a dataset returned, containing 1 month worth of meaned temperature and sea surface height data:

In [12]:
clim_mean  # uncomment to print data object summary

<xarray.Dataset> Size: 7GB
Dimensions:      (y_dim: 1207, x_dim: 1442, z_dim: 75, month: 12)
Coordinates:
  * y            (y_dim) float64 10kB 0.0 1.0 2.0 ... 1.205e+03 1.206e+03
  * x            (x_dim) float64 12kB 0.0 1.0 2.0 ... 1.44e+03 1.441e+03
    longitude    (y_dim, x_dim) float32 7MB ...
    latitude     (y_dim, x_dim) float32 7MB ...
    depth_0      (z_dim, y_dim, x_dim) float64 1GB 0.5 0.5 ... 5.902e+03
  * month        (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: y_dim, x_dim, z_dim
Data variables:
    temperature  (month, z_dim, y_dim, x_dim) float32 6GB dask.array<chunksize=(1, 19, 302, 361), meta=np.ndarray>

### Create multiyear averages for seasons

Calculates the mean over a specified period and groups the data by year-period

data['t_dim'] = data['time']

In [75]:
# Create a list of seasons and years
seasons = [(3, 5), (6, 9), (10, 11), (12, 2)]
data_years = list(data[f"time.year"].data)


In [89]:
# Create a range of dates
date_ranges = []
for y in sorted(set(data_years)):
    y = int(y)
    for period in seasons:
        start = period[0]
        end = period[1]
        begin_date = cftime.Datetime360Day(y, start, 1,0,0,0,0, has_year_zero=True)
        if start > end:
            end_date = cftime.Datetime360Day(y + 1, end, 30,0,0,0,0, has_year_zero=True)
        else:
            end_date = cftime.Datetime360Day(y, end, 30, 0,0,0,0, has_year_zero=True)
        date_ranges.append((begin_date, end_date))

Separate and concat datasets by year periods

In [92]:
datasets = []
year_index = []
month_index = []
for date_range in date_ranges:
    sel_args = {f"{time_dim}": slice(date_range[0], date_range[1])}
    filtered = data.sel(**sel_args)
    datasets.append(filtered)
    year_index = year_index + ([date_range[0].year] * filtered.sizes['t_dim'])
    month_label = f"{calendar.month_abbr[date_range[0].month]}-{calendar.month_abbr[date_range[1].month]}"
    month_index = month_index + ([month_label] * filtered.sizes['t_dim'])

filtered = xr.concat(datasets, dim=time_dim)
filtered = filtered.drop_vars('t_dim')

In [93]:
period_idx = pd.MultiIndex.from_arrays([year_index, month_index], names=("year", "period"))
filtered.coords["year_period"] = ('t_dim', period_idx)

In [94]:
clim_multiyear = xr.Dataset()
for var_name, da in filtered.data_vars.items():
    da_mean = da.groupby("year_period").mean(dim=time_dim, skipna=True)
    clim_multiyear[f"{var_name}"] = da_mean

In [95]:
clim_multiyear

<xarray.Dataset> Size: 137GB
Dimensions:      (y_dim: 1207, x_dim: 1442, z_dim: 75, year_period: 260)
Coordinates:
  * y            (y_dim) float64 10kB 0.0 1.0 2.0 ... 1.205e+03 1.206e+03
  * x            (x_dim) float64 12kB 0.0 1.0 2.0 ... 1.44e+03 1.441e+03
    longitude    (y_dim, x_dim) float32 7MB 72.75 73.0 73.25 ... 73.01 73.0 73.0
    latitude     (y_dim, x_dim) float32 7MB -89.5 -89.5 -89.5 ... 50.0 50.0
    depth_0      (z_dim, y_dim, x_dim) float64 1GB 0.5 0.5 ... 5.902e+03
  * year_period  (year_period) object 2kB MultiIndex
  * year         (year_period) int64 2kB 1950 1950 1950 1950 ... 2014 2014 2014
  * period       (year_period) object 2kB 'Mar-May' 'Jun-Sep' ... 'Dec-Feb'
Dimensions without coordinates: y_dim, x_dim, z_dim
Data variables:
    temperature  (year_period, z_dim, y_dim, x_dim) float32 136GB dask.array<chunksize=(1, 19, 302, 361), meta=np.ndarray>